In [1]:
import pandas as pd
import numpy as np
import json
import folium
from datetime import datetime
import os

import yaml

In [2]:
from data_cleaning.json_manipulation import load_json, trim_json_to_locations, gather_bajs_locations, get_bikes_in_stations, calculate_bike_changes, create_changes_columns
from visualization.station_map import populate_map_with_stations

In [3]:
config = yaml.safe_load(open("../config.YAML"))

In [4]:
with open("../data/test_JSON_data/bajs_10-11-2025_15-11-31.json") as json_data:
    day_one = json.load(json_data)
    json_data.close()

In [5]:
day_one

{'countries': [{'lat': 45.8049,
   'lng': 15.9467,
   'zoom': 10,
   'name': 'Bajs Zagreb (Croatia)',
   'hotline': '0038515494477',
   'domain': 'hd',
   'language': 'hr',
   'email': 'bajs@zagreb.hr',
   'timezone': 'Europe/Berlin',
   'currency': 'EUR',
   'country_calling_code': '+385',
   'system_operator_address': 'Sustav javnih bicikala d.o.o., Prisavlje 2, 10000 Zagreb',
   'country': 'HR',
   'country_name': 'Croatia',
   'terms': 'https://bajs.zagreb.hr/hr/uvjetikoristenja/',
   'policy': 'https://bajs.zagreb.hr/hr/pravila-privatnosti/',
   'website': 'https://bajs.zagreb.hr',
   'show_bike_types': False,
   'show_bike_type_groups': False,
   'show_free_racks': False,
   'booked_bikes': 0,
   'set_point_bikes': 2000,
   'available_bikes': 1812,
   'capped_available_bikes': False,
   'no_registration': False,
   'pricing': 'https://bajs.zagreb.hr/hr/',
   'vat': '25',
   'faq_url': 'https://bajs.zagreb.hr',
   'store_uri_android': 'https://play.google.com/store/apps/details?id

In [6]:
trim_json_to_locations(day_one)

[{'uid': 556632908,
  'lat': 45.825299,
  'lng': 16.109747,
  'bike': False,
  'name': 'AUTOBUSNI TERMINAL SESVETE',
  'address': None,
  'spot': True,
  'number': 21200,
  'booked_bikes': 0,
  'bikes': 5,
  'bikes_available_to_rent': 5,
  'active_place': 1,
  'bike_racks': 8,
  'free_racks': 3,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': 'free',
  'bike_numbers': ['800463', '800020', '801638', '801088', '801072'],
  'bike_types': {'196': 4, '409': 1},
  'place_type': '0',
  'rack_locks': False},
 {'uid': 556633495,
  'lat': 45.842797,
  'lng': 15.975421,
  'bike': False,
  'name': 'MIHALJEVAC OKRETIŠTE',
  'address': None,
  'spot': True,
  'number': 21201,
  'booked_bikes': 0,
  'bikes': 11,
  'bikes_available_to_rent': 11,
  'active_place': 1,
  'bike_racks': 12,
  'free_racks': 1,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': 'free',
  'bike_numbers': ['800480',
   '800247',
   '800185',
 

In [7]:
locations = gather_bajs_locations(trim_json_to_locations(day_one))
locations.head()

,uid,name,lat,lng,no_racks
0,556632908,AUTOBUSNI TERMINAL SESVETE,45.825299,16.109747,8
1,556633495,MIHALJEVAC OKRETIŠTE,45.842797,15.975421,12
2,556634009,ROTOR REMETINEC,45.776795,15.953339,10
3,556637365,ADMIRAL HOTEL,45.795083,15.919185,20
4,556689563,ZAGREBAČKI VELESAJAM,45.777560,15.969703,20


In [8]:
locations.shape
# 165 different stations in Zagreb

(165, 5)

Thunderforest.Neighbourhood
OSMBright
Esri.WorldTopoMap
CartoDB.Voyager

In [9]:
fig = folium.Figure(width = 1100, height = 500)

m = folium.Map(
    location=(45.8109, 16.0097),
    min_zoom=11,
    tiles = "Esri.WorldTopoMap"
)

fig.add_child(m)

In [10]:
m = populate_map_with_stations(m, locations)


In [11]:
m

## Individual bikes at individual times

In [12]:
first_dt = datetime.strptime("2025/11/10 15:11:31",
    "%Y/%m/%d %H:%M:%S")

first_dt

datetime.datetime(2025, 11, 10, 15, 11, 31)

In [13]:
time1 = get_bikes_in_stations(time = first_dt,
    loc_list = trim_json_to_locations(day_one))

time1.sort_values("uid").head()

,uid,time,bikes_at_station
0,556632908,2025-11-10 15:11:31,"[800463, 800020, 801638, 801088, 801072]"
1,556633495,2025-11-10 15:11:31,"[800480, 800247, 800185, 800183, 800106, 80005..."
2,556634009,2025-11-10 15:11:31,"[800220, 800201, 800129, 800079, 801644, 80155..."
3,556637365,2025-11-10 15:11:31,"[801920, 801915, 800748, 800640, 800625]"
4,556689563,2025-11-10 15:11:31,"[801846, 801835, 801548, 801542, 801000, 80099..."


In [14]:
# For testing purposes, we'll use this data as a second timepoint even though it's been more than a day
with open("../data/test_JSON_data/bajs_12-11-2025_09-30-11.json") as json_data:
    day_two = json.load(json_data)
    json_data.close()

In [15]:
second_dt = datetime.strptime("2025/11/10 16:30:00",
    "%Y/%m/%d %H:%M:%S")

time2 = get_bikes_in_stations(time = second_dt,
    loc_list = trim_json_to_locations(day_two))

time2.sort_values("uid").head()

,uid,time,bikes_at_station
0,556632908,2025-11-10 16:30:00,"[800479, 800463, 800282, 801088]"
1,556633495,2025-11-10 16:30:00,"[800480, 800185, 800052, 801489, 801321]"
2,556634009,2025-11-10 16:30:00,"[800274, 801882, 801360, 801296, 801022]"
3,556637365,2025-11-10 16:30:00,"[800344, 800033, 801920, 801915, 801881, 80164..."
4,556689563,2025-11-10 16:30:00,"[800485, 800329, 800233, 800230, 800223, 80019..."


In [16]:
more_times = pd.concat([time1, time2])
# for each uid-time, what is the closest observed time before the one in this row?
more_times["lag1_time"] = more_times.groupby(["uid"])["time"].shift(1)

more_times

previous_bikes = more_times.merge(more_times.drop(["lag1_time"], axis = 1), # so we don't get this as an additional column
    how = "left",
    left_on = ["uid", "lag1_time"],
    right_on = ["uid", "time"],
    suffixes = (None, "_lag1")).drop(
        ["time_lag1"], axis = 1
    )



previous_bikes

,uid,time,bikes_at_station,lag1_time,bikes_at_station_lag1
0,556632908,2025-11-10 15:11:31,"[800463, 800020, 801638, 801088, 801072]",NaT,NaN
1,556633495,2025-11-10 15:11:31,"[800480, 800247, 800185, 800183, 800106, 80005...",NaT,NaN
2,556634009,2025-11-10 15:11:31,"[800220, 800201, 800129, 800079, 801644, 80155...",NaT,NaN
3,556637365,2025-11-10 15:11:31,"[801920, 801915, 800748, 800640, 800625]",NaT,NaN
4,556689563,2025-11-10 15:11:31,"[801846, 801835, 801548, 801542, 801000, 80099...",NaT,NaN
...,...,...,...,...,...
326,585913527,2025-11-10 16:30:00,"[800278, 800101, 801656, 801228, 801046, 80077...",2025-11-10 15:11:31,[]
327,585913605,2025-11-10 16:30:00,"[801717, 801191, 801157, 800603, 800601, 800517]",2025-11-10 15:11:31,[801963]
328,585913702,2025-11-10 16:30:00,"[800347, 800115, 801866, 801826, 801795, 80166...",2025-11-10 15:11:31,"[800293, 800219, 800191, 800068, 801907, 80175..."
329,586256300,2025-11-10 16:30:00,"[800144, 801889, 801867, 801457, 801066, 80095...",2025-11-10 15:11:31,"[800318, 800144, 801889, 801867, 801169, 80116..."


In [17]:
calculate_bike_changes(previous_bikes["bikes_at_station"], previous_bikes["bikes_at_station_lag1"])

{'changes': 0       NaN
 1       NaN
 2       NaN
 3       NaN
 4       NaN
        ... 
 326     8.0
 327     7.0
 328    31.0
 329     5.0
 330     5.0
 Length: 331, dtype: float64,
 'incoming': 0       NaN
 1       NaN
 2       NaN
 3       NaN
 4       NaN
        ... 
 326     0.0
 327     1.0
 328    15.0
 329     3.0
 330     0.0
 Length: 331, dtype: float64,
 'outgoing': 0       NaN
 1       NaN
 2       NaN
 3       NaN
 4       NaN
        ... 
 326     8.0
 327     6.0
 328    16.0
 329     2.0
 330     5.0
 Length: 331, dtype: float64}

## Load all data from folder and calculate changes

In [18]:
raw_files_directory = "../data/" + config["data"]["raw_json_folder"]

raw_files = os.listdir(raw_files_directory)
raw_files

['bajs_10-11-2025_17-04-53.json',
 'bajs_10-11-2025_18-24-00.json',
 'bajs_10-11-2025_19-21-13.json',
 'bajs_10-11-2025_20-15-53.json',
 'bajs_10-11-2025_21-02-43.json',
 'bajs_10-11-2025_22-14-02.json',
 'bajs_10-11-2025_23-01-43.json',
 'bajs_11-11-2025_00-15-08.json',
 'bajs_11-11-2025_01-13-15.json',
 'bajs_11-11-2025_02-05-07.json',
 'bajs_11-11-2025_03-17-33.json',
 'bajs_11-11-2025_04-00-29.json',
 'bajs_11-11-2025_05-04-41.json',
 'bajs_11-11-2025_06-27-27.json',
 'bajs_11-11-2025_07-19-42.json',
 'bajs_11-11-2025_08-17-33.json',
 'bajs_11-11-2025_09-06-29.json',
 'bajs_11-11-2025_10-13-20.json',
 'bajs_11-11-2025_11-06-31.json',
 'bajs_11-11-2025_12-24-02.json',
 'bajs_11-11-2025_13-00-30.json',
 'bajs_11-11-2025_14-08-36.json',
 'bajs_11-11-2025_15-00-39.json',
 'bajs_11-11-2025_16-29-45.json',
 'bajs_11-11-2025_17-04-53.json',
 'bajs_11-11-2025_18-05-53.json',
 'bajs_11-11-2025_19-03-15.json',
 'bajs_11-11-2025_20-05-27.json',
 'bajs_11-11-2025_21-09-35.json',
 'bajs_11-11-2

In [19]:
raw_files[0][5:-5]

'10-11-2025_17-04-53'

In [20]:
#TODO - extracting from first to last number instead of absolute position?

datetime.strptime(raw_files[0][5:-5], "%d-%m-%Y_%H-%M-%S")

datetime.datetime(2025, 11, 10, 17, 4, 53)

In [21]:
scraped_datetimes = [datetime.strptime(filename[5:-5], "%d-%m-%Y_%H-%M-%S") for filename in raw_files]
scraped_datetimes

[datetime.datetime(2025, 11, 10, 17, 4, 53),
 datetime.datetime(2025, 11, 10, 18, 24),
 datetime.datetime(2025, 11, 10, 19, 21, 13),
 datetime.datetime(2025, 11, 10, 20, 15, 53),
 datetime.datetime(2025, 11, 10, 21, 2, 43),
 datetime.datetime(2025, 11, 10, 22, 14, 2),
 datetime.datetime(2025, 11, 10, 23, 1, 43),
 datetime.datetime(2025, 11, 11, 0, 15, 8),
 datetime.datetime(2025, 11, 11, 1, 13, 15),
 datetime.datetime(2025, 11, 11, 2, 5, 7),
 datetime.datetime(2025, 11, 11, 3, 17, 33),
 datetime.datetime(2025, 11, 11, 4, 0, 29),
 datetime.datetime(2025, 11, 11, 5, 4, 41),
 datetime.datetime(2025, 11, 11, 6, 27, 27),
 datetime.datetime(2025, 11, 11, 7, 19, 42),
 datetime.datetime(2025, 11, 11, 8, 17, 33),
 datetime.datetime(2025, 11, 11, 9, 6, 29),
 datetime.datetime(2025, 11, 11, 10, 13, 20),
 datetime.datetime(2025, 11, 11, 11, 6, 31),
 datetime.datetime(2025, 11, 11, 12, 24, 2),
 datetime.datetime(2025, 11, 11, 13, 0, 30),
 datetime.datetime(2025, 11, 11, 14, 8, 36),
 datetime.dateti

In [22]:
data_timepoints = [
    trim_json_to_locations(load_json(raw_files_directory + "/" + json_file))
    for json_file
    in raw_files
]

len(data_timepoints)

141

In [23]:
bike_data = None

for i in range(len(data_timepoints)):
    timepoint_bikes = get_bikes_in_stations(time = scraped_datetimes[i],
        loc_list = data_timepoints[i])

    if i == 0: 
        bike_data = timepoint_bikes
    else:
        bike_data = pd.concat([bike_data, timepoint_bikes])

bike_data

,uid,time,bikes_at_station
0,556632908,2025-11-10 17:04:53,"[800463, 800020, 801638, 801088, 800874]"
1,556633495,2025-11-10 17:04:53,"[800480, 800247, 800185, 800183, 800106, 80005..."
2,556634009,2025-11-10 17:04:53,"[800220, 800201, 800129, 800079, 801882, 80164..."
3,556637365,2025-11-10 17:04:53,"[800345, 800193, 801920, 801915, 801881, 80164..."
4,556689563,2025-11-10 17:04:53,"[801846, 801548, 801347, 801000, 800817, 80062..."
...,...,...,...
163,585913605,2025-11-16 13:10:37,"[800370, 801928, 801796, 801486, 801278, 80115..."
164,585913702,2025-11-16 13:10:37,"[801866, 800910]"
165,586256300,2025-11-16 13:10:37,"[800452, 800144, 801889, 801840, 801558, 80109..."
166,586273004,2025-11-16 13:10:37,"[800457, 801935, 801924, 801917, 801851, 80149..."


In [24]:
bike_data = create_changes_columns(bike_data)
bike_data

,uid,time,bikes_at_station,lag1_time,bikes_at_station_lag1,changes,incoming,outgoing
0,556632908,2025-11-10 17:04:53,"[800463, 800020, 801638, 801088, 800874]",NaT,NaN,NaN,NaN,NaN
1,556633495,2025-11-10 17:04:53,"[800480, 800247, 800185, 800183, 800106, 80005...",NaT,NaN,NaN,NaN,NaN
2,556634009,2025-11-10 17:04:53,"[800220, 800201, 800129, 800079, 801882, 80164...",NaT,NaN,NaN,NaN,NaN
3,556637365,2025-11-10 17:04:53,"[800345, 800193, 801920, 801915, 801881, 80164...",NaT,NaN,NaN,NaN,NaN
4,556689563,2025-11-10 17:04:53,"[801846, 801548, 801347, 801000, 800817, 80062...",NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
23529,585913605,2025-11-16 13:10:37,"[800370, 801928, 801796, 801486, 801278, 80115...",2025-11-16 12:14:20,"[800370, 801928, 801796, 801486, 801278, 80115...",0.0,0.0,0.0
23530,585913702,2025-11-16 13:10:37,"[801866, 800910]",2025-11-16 12:14:20,"[801866, 801566, 800989, 800910]",2.0,0.0,2.0
23531,586256300,2025-11-16 13:10:37,"[800452, 800144, 801889, 801840, 801558, 80109...",2025-11-16 12:14:20,"[800452, 800144, 801889, 801840, 801558, 80109...",0.0,0.0,0.0
23532,586273004,2025-11-16 13:10:37,"[800457, 801935, 801924, 801917, 801851, 80149...",2025-11-16 12:14:20,"[800457, 801935, 801924, 801917, 801851, 80149...",0.0,0.0,0.0


## Total changes on map

In [25]:
# For each station, sum up all changes through time
# Also, weight changes by rack size

total_changes = bike_data.groupby("uid").sum(["changes"])
total_changes = total_changes.rename({"changes":"total_changes"}, axis = 1)
total_changes

,total_changes,incoming,outgoing
uid,,,
556632908,37.0,21.0,16.0
556633495,49.0,23.0,26.0
556634009,265.0,125.0,140.0
556637365,225.0,112.0,113.0
556689563,734.0,371.0,363.0
...,...,...,...
585913605,112.0,60.0,52.0
585913702,133.0,65.0,68.0
586256300,24.0,13.0,11.0


In [26]:
locations_changes = locations.join(total_changes, on = "uid")
locations_changes["weighted_changes"] = locations_changes["total_changes"] / locations_changes["no_racks"]
locations_changes

,uid,name,lat,lng,no_racks,total_changes,incoming,outgoing,weighted_changes
0,556632908,AUTOBUSNI TERMINAL SESVETE,45.825299,16.109747,8,37.0,21.0,16.0,4.625000
1,556633495,MIHALJEVAC OKRETIŠTE,45.842797,15.975421,12,49.0,23.0,26.0,4.083333
2,556634009,ROTOR REMETINEC,45.776795,15.953339,10,265.0,125.0,140.0,26.500000
3,556637365,ADMIRAL HOTEL,45.795083,15.919185,20,225.0,112.0,113.0,11.250000
4,556689563,ZAGREBAČKI VELESAJAM,45.777560,15.969703,20,734.0,371.0,363.0,36.700000
...,...,...,...,...,...,...,...,...,...
160,585913527,GUPČEVA ZVIJEZDA,45.827987,15.979160,10,103.0,53.0,50.0,10.300000
161,585913605,PETROVA UL. - UL. IVANA ZAJCA,45.817526,15.997421,10,112.0,60.0,52.0,11.200000
162,585913702,KORANSKA UL.,45.802365,15.968013,10,133.0,65.0,68.0,13.300000
163,586256300,GAJNICE UL. - GRINTAVEČKA UL.,45.816342,15.872686,10,24.0,13.0,11.0,2.400000


In [27]:
fig = folium.Figure(width = 1100, height = 500)

m = folium.Map(
    location=(45.8109, 16.0097),
    min_zoom=11,
    tiles = "Esri.WorldTopoMap"
)

fig.add_child(m)

In [28]:
m = populate_map_with_stations(m, 
                               locations_changes, 
                               metric_size = "total_changes", 
                               metric_tooltip_name = "Bike changes: ",
                               no_bins = 12)

In [29]:
m

We can see that the number of racks DOESN'T necessarily mean that this is the maximum amount of bikes that can be at a station at the same time. Maybe they're counted if they're in the near vicinity?  

Still, the no_racks is still useful as an approximation of how large/frequently used the station is planned to be.

### Tables of usage
Most and least used total

In [39]:
locations_changes.sort_values("total_changes",  ascending = False).head()

,uid,name,lat,lng,no_racks,total_changes,incoming,outgoing,weighted_changes
26,556723550,TRG KRALJA TOMISLAVA,45.805057,15.978300,57,992.0,479.0,513.0,17.403509
18,556702069,S.D. CVJETNO NASELJE,45.790564,15.961748,30,867.0,439.0,428.0,28.900000
4,556689563,ZAGREBAČKI VELESAJAM,45.777560,15.969703,20,734.0,371.0,363.0,36.700000
58,579364165,FILOZOFSKI FAKULTET,45.796235,15.970371,20,629.0,308.0,321.0,31.450000
12,556696500,S.D. STJEPAN RADIĆ,45.784832,15.949303,20,577.0,296.0,281.0,28.850000


In [40]:
locations_changes.sort_values("total_changes",  ascending = True).head()

,uid,name,lat,lng,no_racks,total_changes,incoming,outgoing,weighted_changes
117,583385565,SD SUTINSKA VRELA,45.817745,15.832586,8,0.0,0.0,0.0,0.0
50,579361783,AUTOBUSNO STAJALIŠTE PRIGORNICA,45.815251,15.848368,10,3.0,1.0,2.0,0.3
131,583391639,CARINARNICA ZAGREB,45.800518,16.057125,8,4.0,0.0,4.0,0.5
155,585911812,RADNIČKA CESTA 228,45.778281,16.054742,10,7.0,5.0,2.0,0.7
71,579365873,PODSUSED OKRETIŠTE,45.815295,15.835378,5,9.0,4.0,5.0,1.8


Most used relative to number of racks provided - possibly could use more racks installed:

In [41]:
locations_changes.sort_values("weighted_changes",  ascending = False).head()

,uid,name,lat,lng,no_racks,total_changes,incoming,outgoing,weighted_changes
87,579368895,PARK KATE ŠOLJIĆ,45.803227,15.942999,7,303.0,149.0,154.0,43.285714
119,583385717,SELSKA CESTA - UL. BLAŽA TROGIRANINA,45.791090,15.944574,10,368.0,181.0,187.0,36.800000
4,556689563,ZAGREBAČKI VELESAJAM,45.777560,15.969703,20,734.0,371.0,363.0,36.700000
142,583393545,PUČKO OTVORENO UČILIŠTE,45.799511,15.970981,14,478.0,243.0,235.0,34.142857
41,573743895,PRERADOVIĆEVA UL.,45.811080,15.974207,10,340.0,163.0,177.0,34.000000
